In [ ]:
'''
In this notebook, we will process the radio maps from the previous notebook and prepare them for NN inference. First, we will use sensors to gather
spectrum power measurements over the power maps. Then we will prepare the sensor measurements and power maps for neural network inference. This will 
include ensuring image resolution size is within the range of the neural network and applying the Log Likelihood Test for Occupancy.
'''
import torch
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
'''
3_CreatingTheDataset
- Sensors
- Subregion Pooling
- Occupancy LLRT
'''

In [ ]:
def make_larger_sensors(sensor_map):
    '''
    This is a function we'll use to help us visualize the sensor locations
    '''
    max_x = sensor_map.size(dim=1)
    max_y = sensor_map.size(dim=0)
    out_sensor_map = torch.zeros(max_y, max_x)
    for y in range(sensor_map.size(dim=0)):
        for x in range(sensor_map.size(dim=1)):
            if sensor_map[y, x]:
                out_sensor_map[y, x] = 1

                back_y = y - 1 >= 0
                for_y = y + 1 < max_y

                back_x = x - 1 >= 0
                for_x = x + 1 < max_x

                if back_y:
                    out_sensor_map[y - 1, x] = 1

                if back_x:
                    out_sensor_map[y, x - 1] = 1

                if for_y:
                    out_sensor_map[y + 1, x] = 1

                if for_x:
                    out_sensor_map[y, x + 1] = 1
    return out_sensor_map

In [ ]:
# LOAD THE POWER MAPS FROM NOTEBOOK 2
power_maps_dBm = torch.load("power_maps.pt")  # dBm
threshold_dBm = -90
num_maps = power_maps_dBm.size(dim=0)

In [ ]:
'''
PUTTING DOWN SPECTRUM SENSORS
Here we'll emulate spectrum sensors across the power maps to generate sensor maps containing spectrum power measurements. There are multiple ways to 
estimate the spectrum power. For this tutorial, however, we'll assume the sensors measure the power exactly and simply measure the mean power of the
subregion.
'''
# Change the number of sensors used per map by editing "num_sensors_per_map"
num_sensors_per_map = 100
sensor_locs = torch.zeros(*power_maps_dBm.shape)
torch.manual_seed(423)

# Generate random sensor locations using the randperm function
for i_map in range(num_maps):
    sensors = torch.randperm(512 * 512).reshape(512, 512) < num_sensors_per_map
    sensor_locs[i_map] = sensors

# Peek at the sensor locations for map 'i_map'. Black dots represent the sensor generated random sensor locations
i_map = 9
viz_sensor_locs = make_larger_sensors(make_larger_sensors(sensor_map=sensor_locs[i_map]))
plt.imshow(-torch.flip(viz_sensor_locs, dims=[0]), vmin=-1, vmax=1, cmap='gray')
plt.axis('off')
plt.title(f'Sensor Locations for Map: {i_map}')
plt.show()

# Generate the sensor maps by sampling the power maps
sensor_maps = power_maps_dBm * sensor_locs

# Peek at sensor locations for map 'i_map'. Black dots are sensors seeing power below the threshold. White dots are sensors seeing power above the 
# threshold. NOTE: The sensors are measuring the power of the spectrum. We are just thresholding them for visualization sake. Notice that the white
# sensor dots line up with where the occupancy circle is.
viz_sensors_above_threshold = make_larger_sensors(make_larger_sensors(sensor_map=((sensor_maps[i_map] > threshold_dBm) * sensor_locs[i_map]).to(torch.float32)))
viz_sensors_below_threshold = make_larger_sensors(make_larger_sensors(sensor_map=((sensor_maps[i_map] < threshold_dBm) * sensor_locs[i_map]).to(torch.float32)))

shrink = 0.6
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(torch.flip(viz_sensors_above_threshold - viz_sensors_below_threshold, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[0].axis("off")
axs[0].set_title('Thresholded\nSensor Measurements')

im = axs[1].imshow(torch.flip(power_maps_dBm[i_map], dims=[0]))
axs[1].axis('off')
axs[1].set_title('Power Map [dBm]')
plt.colorbar(im, ax=axs[1], shrink=shrink)

occupancy_map = power_maps_dBm[i_map] > threshold_dBm
axs[2].imshow(torch.flip(occupancy_map, dims=[0]), interpolation='nearest')
axs[2].axis("off")
axs[2].set_title('Occupancy Map')
plt.show()

In [ ]:
'''
SUBREGION POOLING
We implement subregion pooling to transform the 512x512 radio maps into 128x128 radio maps. To do this, we'll separate the 'full resolution' 512x512
radio maps into non-overlapping 4x4 subregions and perform a pooling operation. We use two pooling operations for the radio maps: power pooling & sensor
pooling. The power pooling is just an average pooling operation and we'll use PyTorch's built-in pooling function. The sensor pooling averages the power 
of all spectrum power measurements in a subregion.
'''
#  POWER MAP POOLING
power_pooling_func = torch.nn.AvgPool2d(kernel_size=4, padding=0, stride=4)  # Initialize PyTorch's built-in Mean Pooling function
pooled_power_maps_dBm = power_pooling_func(10**(power_maps_dBm/10))  # Need to average in mW
pooled_power_maps_dBm = 10 * torch.log10(pooled_power_maps_dBm) # Convert the pooled power maps to log domain [dBm]
print(f'Full-Res Power Maps Shape: {power_maps_dBm.shape}, Pooled Power Maps Shape: {pooled_power_maps_dBm.shape}')

# Generate pooled sensor map
def sensor_pool(sensor_maps, kernel_size=4):
    '''
    This function is similar to Avg2DPool, but instead of averaging every value in a kernel, it only averages the non-zero values. This allows it to
    average all the sensor measurements in a subregion.
    '''
    num_maps, ysize, xsize = sensor_maps.shape
    pooled_sensor_map = torch.zeros(num_maps, ysize // kernel_size, xsize // kernel_size)
    for i_map in range(num_maps):
        for iypool in range(ysize // kernel_size):
            for ixpool in range(xsize // kernel_size):
                subregion = sensor_maps[i_map, kernel_size*iypool:kernel_size*(iypool+1), kernel_size*ixpool:kernel_size*(ixpool+1)].clone() # Grab subregion
                num_sensors_in_subregion = torch.count_nonzero(subregion) # Count the number of sensors in the subregion
                if num_sensors_in_subregion: # If there are sensors in the subregion
                    subregion[subregion != 0] = 10 ** (subregion[subregion != 0] / 10) # Convert subregion sensor values to mW
                    pooled_sensor_map[i_map, iypool, ixpool] = torch.sum(subregion) / num_sensors_in_subregion # Average the sensor measurements
                    pooled_sensor_map[i_map, iypool, ixpool] = 10 * torch.log10(pooled_sensor_map[i_map, iypool, ixpool]) # Convert back to dBm

    return pooled_sensor_map

pooled_sensor_maps = sensor_pool(sensor_maps=sensor_maps, kernel_size=4)


In [ ]:
#  PEEK AT THE POOLED POWER AND SENSOR MAPS
#  Choose which power map and sensor map pair to viz by editing "i_map"
i_map = 2

#  POOLED POWER MAP PEEK
vmin = min([power_maps_dBm[i_map].min(), pooled_power_maps_dBm[i_map].min()])
vmax = max([power_maps_dBm[i_map].max(), pooled_power_maps_dBm[i_map].max()])
shrink = 0.48
fig, axs = plt.subplots(1, 2)
im = axs[0].imshow(torch.flip(power_maps_dBm[i_map], dims=[0]), vmin=vmin, vmax=vmax)
axs[0].axis('off')
axs[0].set_title('Full-Res Power Map [dBm]')
plt.colorbar(im, ax=axs[0], shrink=shrink)

im = axs[1].imshow(torch.flip(pooled_power_maps_dBm[i_map], dims=[0]), vmin=vmin, vmax=vmax)
axs[1].axis('off')
axs[1].set_title('Pooled Power Map [dBm]')
plt.colorbar(im, ax=axs[1], shrink=shrink)
plt.tight_layout()
plt.show()

#  POOLED SENSOR MAP PEEK
viz_sensors_above_threshold = make_larger_sensors(make_larger_sensors(sensor_map=((sensor_maps[i_map] > threshold_dBm) * sensor_locs[i_map]).to(torch.float32)))
viz_sensors_below_threshold = make_larger_sensors(make_larger_sensors(sensor_map=((sensor_maps[i_map] < threshold_dBm) * sensor_locs[i_map]).to(torch.float32)))

pooled_viz_sensors_above_threshold = ((pooled_sensor_maps[i_map] > threshold_dBm) * (pooled_sensor_maps[i_map] != 0)).to(torch.float32)
pooled_viz_sensors_below_threshold = ((pooled_sensor_maps[i_map] < threshold_dBm) * (pooled_sensor_maps[i_map] != 0)).to(torch.float32)

print(f'Full-Res Sensor Maps Shape: {sensor_maps.shape}, Pooled Sensor Maps Shape: {pooled_sensor_maps.shape}')
fig, axs = plt.subplots(1, 2)
axs[0].imshow(torch.flip(viz_sensors_above_threshold - viz_sensors_below_threshold, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[0].axis("off")
axs[0].set_title('Sensor Measurements')

axs[1].imshow(torch.flip(pooled_viz_sensors_above_threshold - pooled_viz_sensors_below_threshold, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[1].axis("off")
axs[1].set_title('Pooled Sensor Measurements')
plt.show()

In [ ]:
'''
GENERATE THE GROUND TRUTH POOLED OCCUPANCY MAPS THAT THE NEURAL NETWORK IS TRYING TO ESTIMATE
'''
pooled_occupancy_maps = (pooled_power_maps_dBm > threshold_dBm)

# Peek the pooled occupancy maps and compare to the pooled power maps
i_map = 2
vmin = min([power_maps_dBm[i_map].min(), pooled_power_maps_dBm[i_map].min()])
vmax = max([power_maps_dBm[i_map].max(), pooled_power_maps_dBm[i_map].max()])
shrink = 0.48

fig, axs = plt.subplots(1, 2)
axs[0].imshow(torch.flip(pooled_occupancy_maps[i_map], dims=[0]), vmin=0, vmax=1, interpolation='nearest')
axs[0].axis('off')
axs[0].set_title('Pooled Occupancy Maps')

im = axs[1].imshow(torch.flip(pooled_power_maps_dBm[i_map], dims=[0]), vmin=vmin, vmax=vmax)
axs[1].axis('off')
axs[1].set_title('Pooled Power Maps [dBm]')
plt.colorbar(im, ax=axs[1], shrink=shrink)
plt.tight_layout()
plt.show()

In [ ]:
'''
LOG LIKELIHOOD RATIO TEST FOR SPECTRUM OCCUPANCY
Lastly, we treat the spectrum occupancy mapping problem as a detection problem where we are trying to decide between two hypotheses:
H0: Unoccupied, or the average spectrum power in the subregion is below the threshold
H1: Occupied, or the average spectrum power in a subregion is equal to or greater than the threshold

The spectrum power sensors measure a power value close to the average power in the subregion they are in. To model this, we say the spectrum power
measurement, m(s) is: m(s) = <f(r)G> + z(s) where 
- <f(r)G> is the average spectrum power in a subregion
- z(s) is the error in representing the <f(r)G> by a sensor measurement. This is modeled as a Gaussian r.v.

Now, we wish to determine whether H0: <f(r)G> < threshold or H1: <f(r)G> >= threshold
We derive the simplified LLR(s) = m(s) - threshold

Finally, we apply the LLR(s) to the sensor measurements. This creates the pooled llr sensor maps that the neural network performs inference over.

NOTE: Applying LLR(s) to each sensor individually and sensor pooling the LLR values is equivalent to pooling the sensor power values and then
applying the LLR to the pooled sensor maps to subregions that contained sensors.
'''
threshold_dBm = -90
threshold_mW = 10**(threshold_dBm / 10)

num_maps, ysize, xsize = pooled_sensor_maps.shape

pooled_llr_sensor_maps = torch.zeros(num_maps, ysize, xsize)
for i_map in range(num_maps):
    for iypool in range(ysize):
        for ixpool in range(xsize):
            if pooled_sensor_maps[i_map, iypool, ixpool]:  # If the subregion contained a sensor (and thus contains a nonzero value)
                mean_sensor_power = 10**(pooled_sensor_maps[i_map, iypool, ixpool] / 10) # Convert the sensor measurement to mW
                pooled_llr_sensor_maps[i_map, iypool, ixpool] = (mean_sensor_power - threshold_mW) * 1e-3 # Apply the LLR in Watts (instead of mW)

# For neural network stability, we also normalize each input map to unit variance
for i_map in range(num_maps):
    pooled_llr_sensor_maps[i_map] /= (pooled_llr_sensor_maps[i_map]).std()

# Peek the pooled llr sensor maps and ensure it matches the pooled sensor maps
i_map = 2
pooled_viz_llr_sensors_H1 = (pooled_llr_sensor_maps[i_map] > 0).to(torch.float32)
pooled_viz_llr_sensors_H0 = (pooled_llr_sensor_maps[i_map] < 0).to(torch.float32)
pooled_viz_sensors_above_threshold = ((pooled_sensor_maps[i_map] > threshold_dBm) * (pooled_sensor_maps[i_map] != 0)).to(torch.float32)
pooled_viz_sensors_below_threshold = ((pooled_sensor_maps[i_map] < threshold_dBm) * (pooled_sensor_maps[i_map] != 0)).to(torch.float32)

fig, axs = plt.subplots(1, 2)
axs[0].imshow(torch.flip(pooled_viz_llr_sensors_H1 - pooled_viz_llr_sensors_H0, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[0].axis("off")
axs[0].set_title('Pooled LLR Sensor Measurements')

axs[1].imshow(torch.flip(pooled_viz_sensors_above_threshold - pooled_viz_sensors_below_threshold, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[1].axis("off")
axs[1].set_title('Pooled Sensor Measurements')
plt.show()

In [ ]:
'''
SAVE THE POOLED OCCUPANCY MAPS AND POOLED LLR SENSOR MAPS FOR NEURAL NETWORK INFERENCE
'''
import os
torch.save(pooled_occupancy_maps, f'{os.getcwd()}/pooled_occupancy_maps.pt')
torch.save(pooled_llr_sensor_maps, f'{os.getcwd()}/pooled_llr_sensor_maps.pt')